In [2]:
!pip install gym_super_mario_bros nes_py
!pip install stable-baselines3 shimmy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
  Created wheel for nes_py: filename=nes_py-8.2.1-cp312-cp312-linux_x86_64.whl size=535724 sha256=c09cf5315b0f7fc2aade4fa0c8666a78fb8d4dc74906ed205900c1347ef3528d
  Stored in directory: /root/.cache/pip/wheels/1f/a7/fa/9b0357f258d2e68bdc71df972e02418bceb02355ac1f365c59
Successfully built nes_py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 6.8 MB/s eta 0:00:00


In [23]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3 import PPO
from nes_py._rom import ROM
from gym_super_mario_bros.smb_env import SuperMarioBrosEnv
import gym

# ==========================================
# PART 1: THE "HACKER FIXES" (Library Patches)
# ==========================================
# 1. Fix ROM memory math errors
def fixed_prg_rom_stop(self):
    return self.prg_rom_start + int(self.prg_rom_size) * 1024
def fixed_chr_rom_stop(self):
    return self.chr_rom_start + int(self.chr_rom_size) * 1024
ROM.prg_rom_stop = property(fixed_prg_rom_stop)
ROM.chr_rom_stop = property(fixed_chr_rom_stop)

# 2. Fix Mario's X-position calculation
def fixed_x_position(self):
    return int(self.ram[0x6d]) * 0x100 + int(self.ram[0x86])
SuperMarioBrosEnv._x_position = property(fixed_x_position)

# ==========================================
# PART 2: THE "TRANSLATOR" (API Fix)
# ==========================================
class MarioCompatibility(gym.Wrapper):
    def reset(self, **kwargs):
        # Swallow the 'seed' argument so Old Mario doesn't crash
        obs = self.env.reset()
        # Return tuple (obs, info) for New AI
        return obs, {}

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        # Return 5 values (obs, reward, terminated, truncated, info)
        return obs, reward, done, False, info

# ==========================================
# PART 3: TRAINING
# ==========================================

# 1. Create Environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# 2. Apply Wrappers
env = GrayScaleObservation(env, keep_dim=True)
env = ResizeObservation(env, shape=84)
env = MarioCompatibility(env) # <--- The API Fix is applied here
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

# 3. Train
print("🚀 Starting Mario Training... (This will take 10-15 minutes)")
model = PPO("CnnPolicy", env, verbose=1, learning_rate=0.0001, n_steps=4096)
model.learn(total_timesteps=50000)
model.save("mario_model")

print("✅ DONE! Mario is trained and saved.")

🚀 Starting Mario Training... (This will take 10-15 minutes)
Using cpu device
Wrapping the env in a VecTransposeImage.


AttributeError: module 'numpy' has no attribute 'bool8'

In [1]:
import numpy as np

# --- 1. THE NUMPY 2.0 PATCH (Fixes "AttributeError: bool8") ---
# We force the old 'bool8' to just be a standard boolean.
# This MUST happen before importing stable_baselines3.
np.bool8 = np.bool_

import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3 import PPO
from nes_py._rom import ROM
from gym_super_mario_bros.smb_env import SuperMarioBrosEnv
import gym

# --- 2. THE LIBRARY PATCHES (Fixes Math & Reset Errors) ---
# Fix ROM memory math
def fixed_prg_rom_stop(self):
    return self.prg_rom_start + int(self.prg_rom_size) * 1024
def fixed_chr_rom_stop(self):
    return self.chr_rom_start + int(self.chr_rom_size) * 1024
ROM.prg_rom_stop = property(fixed_prg_rom_stop)
ROM.chr_rom_stop = property(fixed_chr_rom_stop)

# Fix Mario X-position
def fixed_x_position(self):
    return int(self.ram[0x6d]) * 0x100 + int(self.ram[0x86])
SuperMarioBrosEnv._x_position = property(fixed_x_position)

# --- 3. THE COMPATIBILITY WRAPPER (Fixes "Too many values to unpack") ---
class MarioCompatibility(gym.Wrapper):
    def reset(self, **kwargs):
        obs = self.env.reset() # Ignore seed
        return obs, {} # Return (obs, info)

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, False, info # Return 5 values

# --- 4. TRAINING LOOP ---
print("🚀 Setting up Mario...")

# Create Env
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# Apply Wrappers
env = GrayScaleObservation(env, keep_dim=True)
env = ResizeObservation(env, shape=84)
env = MarioCompatibility(env) # Apply our translator
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

# Train
print("🍄 Training Started! (This will take ~10 mins)")
model = PPO("CnnPolicy", env, verbose=1, learning_rate=0.0001, n_steps=4096)
model.learn(total_timesteps=50000)
model.save("mario_model")

print("✅ DONE! Mario is trained.")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


🚀 Setting up Mario...


/usr/local/lib/python3.12/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.12/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It 

🍄 Training Started! (This will take ~10 mins)
Using cpu device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.12/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------
| time/              |      |
|    fps             | 139  |
|    iterations      | 1    |
|    time_elapsed    | 29   |
|    total_timesteps | 4096 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 63         |
|    iterations           | 2          |
|    time_elapsed         | 128        |
|    total_timesteps      | 8192       |
| train/                  |            |
|    approx_kl            | 0.03172444 |
|    clip_fraction        | 0.194      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.92      |
|    explained_variance   | -0.00141   |
|    learning_rate        | 0.0001     |
|    loss                 | 3.75       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.00104   |
|    value_loss           | 21         |
----------------------------------------
-----------------------------------------
| time/   

In [2]:
import imageio
import numpy as np
from stable_baselines3 import PPO

# --- Re-define the Wrapper for the Test Environment ---
class MarioCompatibility(gym.Wrapper):
    def reset(self, **kwargs):
        obs = self.env.reset()
        return obs, {}
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, False, info

# 1. Load the Model we just trained
model = PPO.load("mario_model")

# 2. Re-build the Environment (Must match training exactly)
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = ResizeObservation(env, shape=84)
env = MarioCompatibility(env) # <--- Don't forget the translator!
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

# 3. Record the Run
print("🎥 Recording Mario's Run...")
obs = env.reset()
images = []

# Let him play for 1000 frames
for i in range(1500):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)

    # Render the original color image for the video
    img = env.envs[0].env.unwrapped.render(mode='rgb_array')
    images.append(img)

    if done:
        obs = env.reset()

env.close()

# 4. Save Video
imageio.mimsave('mario_result.mp4', [np.array(img) for img in images], fps=60)
print("✅ Video saved: mario_result.mp4")

/usr/local/lib/python3.12/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.12/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.12/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium envir

🎥 Recording Mario's Run...
✅ Video saved: mario_result.mp4


In [3]:
import imageio
import numpy as np
import cv2 # We use OpenCV to resize the tiny AI images
from stable_baselines3 import PPO

# --- Re-define Wrapper (Just like before) ---
class MarioCompatibility(gym.Wrapper):
    def reset(self, **kwargs):
        obs = self.env.reset()
        return obs, {}
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, False, info

# 1. Load Model
model = PPO.load("mario_model")

# 2. Setup Environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = ResizeObservation(env, shape=84)
env = MarioCompatibility(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

# 3. Record "Brain View"
print("🎥 Recording AI Input (Grayscale)...")
obs = env.reset()
images = []

for i in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)

    # TRICK: Instead of asking the emulator for color,
    # we take the EXACT image the AI is looking at right now.
    # obs shape is (1, 84, 84, 4). We grab the last frame [-1].
    ai_view = obs[0, :, :, -1]

    # It's tiny (84x84), so we make it big (500x500) so you can see it
    ai_view_big = cv2.resize(ai_view.astype(np.uint8), (500, 500), interpolation=cv2.INTER_NEAREST)

    images.append(ai_view_big)

    if done:
        obs = env.reset()

env.close()

# 4. Save
imageio.mimsave('mario_brain_view.mp4', images, fps=30)
print("✅ Video saved: mario_brain_view.mp4")

🎥 Recording AI Input (Grayscale)...


✅ Video saved: mario_brain_view.mp4
